# Docs

Generate documentation for the project. Extract MD cells from all the notebooks
and turn them into a doc site powered by [Vitepress](https://vitepress.dev/)
hosted on Github Pages.

In [48]:
//| export

import path from "node:path";
import type { Config } from "jurassic/config.ts";
import { getExportedDefinitions } from "jurassic/utils.ts";
import {
  getCellOutput,
  getNotebooksToProcess,
  loadNb,
} from "jurassic/notebooks.ts";
import type { Cell } from "jurassic/notebooks.ts";

The following cells are considered document cells:

- all markdown cells
- code cells that do not have `//| export` directive: we want to display code
  from these cells as part of the document alongside their output; this is
  useful for demoing functionality and showing tests results?

In [49]:
//| export

export const isDocCell = (cell: Cell): boolean =>
  cell.cell_type === "markdown" ||
  (cell.cell_type === "code" &&
    cell.source.length > 0 &&
    !cell.source[0].match(/^\/\/\S*|\S*export/gi));

In [50]:
import { assertEquals } from "jsr:@std/assert";

Deno.test("isDocCell", () => {
  assertEquals(
    isDocCell({ cell_type: "code", source: ["//| export\n"] }),
    false,
  );
  assertEquals(
    isDocCell({ cell_type: "code", source: ["const c = 1;"] }),
    true,
  );
  assertEquals(
    isDocCell({ cell_type: "code", source: ["//|export\n"] }),
    false,
  );
  assertEquals(
    isDocCell({
      cell_type: "code",
      source: ['import path from "node:path";\n'],
    }),
    true,
  );
  assertEquals(
    isDocCell({
      cell_type: "markdown",
      source: ["# showing //| export\n"],
    }),
    true,
  );
});

isDocCell ... ok (0ms)

ok | 1 passed | 0 failed (0ms)


In [51]:
//| export

const wrapCode = (code: string): string => "```typescript\n" + code + "\n```\n";

export const processCell = (cell: Cell): string => {
  if (cell.cell_type === "markdown") {
    // markdown cells - just show content directly
    return cell.source.join("");
  }

  if (cell.cell_type === "code") {
    // code cells - show code and output
    const code = cell.source.join("");
    const exports = getExportedDefinitions(code);

    if (!exports) {
      return (
        wrapCode(code) +
        "\n" +
        getCellOutput(cell)
      );
    }

    return exports.reduce(
      (acc, e) => acc + "\n" + `## ${e.name}` + "\n\n" + wrapCode(e.signature),
      "",
    );
  }

  return "";
};

In [52]:
await Deno.jupyter.display(
  {
    "text/markdown": processCell({
      cell_type: "code",
      source: ['export const c = () => return "c";'],
    }),
  },
  { raw: true },
);


## c

```typescript
const c = () => any
```


In [53]:
//| export

const moduleHeader = (): string => `
---
outline: deep
---
`;

const processNb = async (
  nbPath: string,
  moduleName: string,
): Promise<string> => {
  // TODO: make use of moduleName
  console.log("Processing notebook", moduleName);
  const nb = await loadNb(nbPath);
  // doc cells only
  return nb.cells.reduce(
    (acc, cell) => acc + "\n\n" + processCell(cell),
    moduleHeader(),
  ).trim();
};

In [54]:
await Deno.jupyter.display(
  {
    "text/markdown": await processNb(
      path.resolve("./export.ipynb"),
      "export.ipynb",
    ),
  },
  { raw: true },
);

Processing notebook export.ipynb


---
outline: deep
---


# Export

Parse notebook and extract exportable code cells into corresponding TS modules
(directives shamelessly copied from `nbdev`)



Helpers for determining if a given line in a cell is a directive. Directives
look like this:

```ts
//| export
```





Determine if a given cell is exportable. "Exportable" means that its contents
will end up in corresponding ts module.





Process notebook - transfer exportable code from cells into ts module





Main export functionality. `exportNb` should work on both individual notebooks
and directories containing notebooks and subdirectories containing more
notebooks 🕳. `notebookPath` is relative to `config.nbsPath`


## exportNb

```typescript
const exportNb = (notebookPath: string, config: Config) => Promise<void>
```


## Tests

Let's test export functionality

# Setup docs project structure

Docs are build using Vitepress. We need to setup some basic scaffolding for it
to work:

- landing page defined in `index.md`

In [55]:
//| export

const indexMd = `
---
# https://vitepress.dev/reference/default-theme-home-page
layout: home

hero:
  name: "Jurassic"
  text: "Jurassic docs"
  tagline: My great project tagline
  actions:
    - theme: brand
      text: Markdown Examples
      link: /markdown-examples
    - theme: alt
      text: API Examples
      link: /api-examples

features:
  - title: Feature A
    details: Lorem ipsum dolor sit amet, consectetur adipiscing elit
  - title: Feature B
    details: Lorem ipsum dolor sit amet, consectetur adipiscing elit
  - title: Feature C
    details: Lorem ipsum dolor sit amet, consectetur adipiscing elit
---
`.trim();

- `package.json` with deps and scripts

In [56]:
//| export

const packageJSON = `
{
  "name": "docs",
  "version": "1.0.0",
  "description": "",
  "main": "index.js",
  "scripts": {
    "docs:dev": "vitepress dev",
    "docs:build": "vitepress build",
    "docs:preview": "vitepress preview"
  },
  "keywords": [],
  "author": "",
  "license": "ISC",
  "devDependencies": {
    "vitepress": "^1.5.0"
  }
}`.trim();

- VitePress config `.vitepress/config.mts`

In [57]:
//| export

const vitePressConfig = (notebooks: string[]): string => {
  const docs = `{
        text: "Docs",
        items: ${
    JSON.stringify(
      // sort by notebook name length for now
      [...notebooks].sort((a, b) => a.length - b.length).map((nb) => ({
        text: nb,
        link: `/${nb.replace(".ipynb", "")}`,
      })),
    )
  },
      },`;
  return `
import { defineConfig } from "vitepress";

// https://vitepress.dev/reference/site-config
export default defineConfig({
  title: "Jurassic",
  description: "Jurassic docs",
  base: "/jurassic/",
  themeConfig: {
    // https://vitepress.dev/reference/default-theme-config
    nav: [
      { text: "Home", link: "/" },
    ],

    search: {
      provider: "local",
    },

    sidebar: [${docs}],

    socialLinks: [
      { icon: "github", link: "https://github.com/vuejs/vitepress" },
    ],
  },
});
`.trim();
};

- `.gitignore`

In [58]:
//| export

const gitIgnore = `
node_modules
.vitepress/dist
.vitepress/cache
`.trim();

- docs flow
  - create docs dir
  - init vitepress inside
    - package.json
    - .gitignore
    - .vitepress dir
- populate .vitepress/config.mts based on settings and layout
- extract md from all notebooks and put in corresponding md files in docs

In [59]:
//| export

export const generateDocs = async (
  notebookPath: string,
  config: Config,
): Promise<void> => {
  const notebooksToProcess: string[] = await getNotebooksToProcess(
    notebookPath,
    config,
  );

  try {
    await Deno.stat(config.docsPath);
    await Deno.remove(config.docsPath, { recursive: true });
  } catch {
    // noop
  }

  // let's go through all notebooks and process them one by one
  for (const notebook of notebooksToProcess) {
    // output module is the same as the input notebook, but with .ts extension
    const outputFile = notebook.replace(".ipynb", ".md");
    // make sure we preserve subdirectories if any
    const outputDir = path.join(config.docsPath, path.dirname(outputFile));
    await Deno.mkdir(outputDir, { recursive: true });
    await Deno.writeTextFile(
      path.join(config.docsPath, outputFile),
      await processNb(path.resolve(config.nbsPath, notebook), notebook),
    );
  }

  const filesToWrite = {
    "index.md": indexMd,
    "package.json": packageJSON,
    ".vitepress/config.mts": vitePressConfig(notebooksToProcess),
    ".gitignore": gitIgnore,
  };

  // create .vitepress directory
  await Deno.mkdir(path.join(config.docsPath, ".vitepress"));

  // Write all files in a loop
  for (const [filename, content] of Object.entries(filesToWrite)) {
    await Deno.writeTextFile(path.join(config.docsPath, filename), content);
  }
};

# Tests

Test doc generator

In [60]:
import { assert } from "jsr:@std/assert";
import { getTestConfig } from "jurassic/config.ts";
import { dirListing } from "jurassic/utils.ts";

Deno.test("generateDocs", async (t) => {
  // set things up, let's recreate mini project structure inside a temp dir
  const td = await Deno.makeTempDir({});

  // recreate nbs dire in temp dir and copy notebooks there
  await Deno.mkdir(`${td}/nbs`),
    Deno.copyFileSync("./export.ipynb", `${td}/nbs/export.ipynb`);
  // recreate submodule directory and copy hello.ipynb to it
  await Deno.mkdir(`${td}/nbs/submodule`),
    Deno.copyFileSync(
      "./submodule/hello.ipynb",
      `${td}/nbs/submodule/hello.ipynb`,
    );

  await t.step("test generateDocs", async () => {
    await generateDocs("./", getTestConfig(td));

    // make sure output modules are created
    Deno.readTextFile(`${td}/docs/package.json`);
    Deno.readTextFile(`${td}/docs/index.md`);
    const vitepressConfig = await Deno.readTextFile(
      `${td}/docs/.vitepress/config.mts`,
    );
    const exportContent = await Deno.readTextFile(`${td}/docs/export.md`);
    const submoduleExportContent = await Deno.readTextFile(
      `${td}/docs/submodule/hello.md`,
    );

    // spot check content inside the output modules
    assert(exportContent.includes("# Export"));
    assert(submoduleExportContent.includes("# Hello"));

    // spot check vitepress config
    assert(vitepressConfig.includes("export.ipynb"));
    assert(vitepressConfig.includes("submodule/hello.ipynb"));

    // pretty print temp directory structure
    await Deno.jupyter.display(
      {
        "text/markdown": "```md\n" + (await dirListing(td)) + "\n```",
      },
      { raw: true },
    );
  });
});

generateDocs ...
  test generateDocs ...Processing notebook submodule/hello.ipynb
Processing notebook export.ipynb


```md
- nbs
  - submodule
    - hello.ipynb
  - export.ipynb
- docs
  - submodule
    - hello.md
  - .vitepress
    - config.mts
  - .gitignore
  - package.json
  - index.md
  - export.md

```

 ok (21ms)
generateDocs ... ok (23ms)

ok | 1 passed (1 step) | 0 failed (25ms)


In [61]:
// import { getConfig } from "jurassic/config.ts";

// await generateDocs("./", await getConfig());

await loadNb("./export.ipynb");

{
  cells: [
    {
      cell_type: "markdown",
      source: [
        "# Export\n",
        "\n",
        "Parse notebook and extract exportable code cells into corresponding TS modules\n",
        "(directives shamelessly copied from `nbdev`)"
      ]
    },
    {
      cell_type: "code",
      source: [
        "//| export\n",
        "\n",
        'import path from "node:path";\n',
        'import { getNotebooksToProcess, loadNb } from "jurassic/notebooks.ts";\n',
        'import type { Config } from "jurassic/config.ts";\n',
        'import type { Cell } from "jurassic/notebooks.ts";'
      ],
      outputs: []
    },
    {
      cell_type: "markdown",
      source: [
        "Helpers for determining if a given line in a cell is a directive. Directives\n",
        "look like this:\n",
        "\n",
        "```ts\n",
        "//| export\n",
        "```"
      ]
    },
    {
      cell_type: "code",
      source: [
        "//| export\n",
        "\n",
        "const isDirective 

In [62]:
"//| export\n".match(/^\/\/\S* export/ig);

[ "//| export" ]